In [203]:
from collections import deque
import numpy as np
from search_problems import Node, GraphSearchProblem
from typing import List, Tuple
from random import randint
import time


In [204]:

def breadth_first_search(problem: GraphSearchProblem)-> Tuple[List[int],int,int]:
    max_frontier_size= 0
    path = []
    
    frontier = deque([Node(None,problem.init_state,None,0)])  # Initialize the frontier with the initial state
    explored = set()  # Initialize an empty set to keep track of explored nodes
    while len(frontier):
        if len(frontier) > max_frontier_size:
            max_frontier_size = len(frontier)
        node = frontier.pop()  # Choose the deepest node in the frontier
        explored.add(node.state)
        if problem.goal_test(node.state):  # Check if the node contains a goal state
            # Return the solution path
            return problem.trace_path(node), len(explored), max_frontier_size
        for action in problem.get_actions(node.state):
            child = problem.get_child_node(node,action)
            
            if child.state not in explored and child not in frontier:
                explored.add(child.state)
                if problem.goal_test(child.state):
                    return problem.trace_path(child), len(explored), max_frontier_size
                frontier.append(child)
    return path, len(explored), max_frontier_size

In [235]:
def bidirectional_search(problem: GraphSearchProblem)-> Tuple[List[int],int,int]:
    """
        Implement a bidirectional search algorithm that takes instances of SimpleSearchProblem (or its derived
        classes) and provides a valid and optimal path from the initial state to the goal state.

        :param problem: instance of SimpleSearchProblem
        :return: path: a list of states (ints) describing the path from problem.init_state to problem.goal_state[0]
                 num_nodes_expanded: number of nodes expanded by the search
                 max_frontier_size: maximum frontier size during search
        """
    max_frontier_size = 0
    num_nodes_expanded = 0
    
    forward_frontier = deque([Node(None,problem.init_state,None,0)])  # Initialize the frontier with the initial state
    backward_frontier = deque([Node(None,problem.goal_states[0],None,0)])  # Initialize the frontier with the initial state
    forward_explored = set()  # Initialize an empty set to keep track of explored nodes
    backward_explored = set()  # Initialize an empty set to keep track of explored nodes
    
    front_discard = dict()
    back_discard = dict()
    while len(forward_frontier) + len(backward_frontier):
        max_frontier_size = max(max_frontier_size,len(forward_frontier)+len(backward_frontier)) 
                
        for _ in range(len(forward_frontier)):
            
            forward_node = forward_frontier.popleft()  # Choose the deepest node in the frontier
        
            forward_explored.add(forward_node.state)
            front_discard[forward_node.state] = forward_node 
        
            num_nodes_expanded +=1  
            
            actions = problem.get_actions(forward_node.state)
        
            for action in actions:
                
                if action[1] not in forward_explored:
                    
                    child = problem.get_child_node(forward_node,action)
                    
                    forward_frontier.append(child)
                    
                    if (child.state in backward_explored):
                        
                        backward = back_discard[child.state]
                        
                        path = problem.trace_path(child)[:-1]
                        path.extend(problem.trace_path(backward,problem.goal_states[0])[::-1])
                        
                        return path, num_nodes_expanded, max_frontier_size
        back_discard.clear()
        
        for _ in range(len(backward_frontier)):
            
            backward_node = backward_frontier.popleft()  # Choose the deepest node in the frontier
        
            backward_explored.add(backward_node.state)
            back_discard[backward_node.state] = backward_node 
        
            num_nodes_expanded +=1  
            
            actions = problem.get_actions(backward_node.state)
        
            for action in actions:
                
                if action[1] not in backward_explored:
                    
                    child = problem.get_child_node(backward_node,action)
                    
                    backward_frontier.append(child)
                    
                    if (child.state in forward_explored):
                        
                        forward = front_discard[child.state]
                        
                       
                        path = problem.trace_path(forward)[:-1]
                        path.extend(problem.trace_path(child,problem.goal_states[0])[::-1])
                        
                        
                        return path, num_nodes_expanded, max_frontier_size
        
        
        front_discard.clear()
        
    return [], num_nodes_expanded, max_frontier_size

In [237]:

E = np.loadtxt('stanford_large_network_facebook_combined.txt', dtype=int)
V = np.unique(E)
goal_states = [randint(0,4035)]
init_state = 0
problem = GraphSearchProblem(goal_states, init_state, V, E)
durs = []
for i in range(1000,4000):

    problem.goal_states = [randint(0,4035)]

    start_time = time.time()
    path, num_nodes_expanded, max_frontier_size = bidirectional_search(problem)
    end_time = time.time()
    if len(path) > 6:
        duration  = end_time - start_time
        correct = problem.check_graph_solution(path)
        print(duration, "seconds to run.",correct,len(path),num_nodes_expanded, max_frontier_size)
        


0.36905741691589355 seconds to run. True 7 6222 45176
0.24960875511169434 seconds to run. True 7 7111 45228
0.24890828132629395 seconds to run. True 7 7109 45133
0.25554966926574707 seconds to run. True 7 6294 45317
0.24862122535705566 seconds to run. True 7 6119 45274
0.35369443893432617 seconds to run. True 7 6578 45195
0.2554028034210205 seconds to run. True 7 7274 45114
0.37865376472473145 seconds to run. True 7 6142 45267
0.28630924224853516 seconds to run. True 7 6514 45240
0.37845468521118164 seconds to run. True 7 6609 45298
0.370851993560791 seconds to run. True 7 6388 45282
0.37032532691955566 seconds to run. True 7 6790 45163
0.24611616134643555 seconds to run. True 7 6409 45164
0.24731802940368652 seconds to run. True 7 6514 45240
0.4399299621582031 seconds to run. True 7 6760 45392
0.2561633586883545 seconds to run. True 7 6644 45299
0.36872076988220215 seconds to run. True 7 6757 45329
0.2506566047668457 seconds to run. True 7 6412 45269
0.2573885917663574 seconds to run.